In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#import scipy.stats as stats
#import statsmodels.api as sm
import re
import plotly.express as px
import requests
from bs4 import BeautifulSoup as BS
from io import StringIO
from shapely.geometry import Point
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
%matplotlib inline

In [10]:
wiki = pd.read_csv('../data/wiki.csv')

In [11]:
wiki

,Unnamed: 0,origin,country,lat,long,depth_km,magnitude,secondary_effects,pde_shaking_deaths,pde_total_deaths,utsu_total_deaths,emdat_total_deaths,deaths,magnitude.1,measure
0,0,1900-05-11 17:23,Japan,38.700,141.100,5,7.0 MJMA,NaN,NaN,NaN,NaN,NaN,NaN,7.0,MJMA
1,1,1900-07-12 06:25,Turkey,40.300,43.100,NaN,5.9 Muk,NaN,NaN,NaN,140.0,NaN,140.0,5.9,Muk
2,2,1900-10-29 09:11,Venezuela,11.000,-66.000,0,7.7 Mw,NaN,NaN,NaN,NaN,NaN,NaN,7.7,Mw
3,3,1901-02-15 00:00,China,26.000,100.100,0,6.5 Ms,NaN,NaN,NaN,NaN,NaN,NaN,6.5,Ms
4,4,1901-03-31 07:11,Bulgaria,43.400,28.700,NaN,6.4 Muk,NaN,NaN,NaN,4.0,NaN,4.0,6.4,Muk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335,1335,2011-03-24 20:25,Burma (see 2011 Burma earthquake),NaN,NaN,NaN,6.8 Mw,NaN,NaN,NaN,NaN,NaN,NaN,6.8,Mw
1336,1336,2011-04-07 14:32,Japan (see April 2011 Miyagi earthquake),38.2,140.0,66.0,7.1 Mw,NaN,NaN,NaN,NaN,NaN,NaN,7.1,Mw
1337,1337,2011-09-18 12:40,India (see 2011 Sikkim earthquake),27.723,88.064,19.7,6.9 Mw,L,NaN,NaN,NaN,NaN,NaN,6.9,Mw
1338,1338,2011-09-23 10:41,Turkey (see 2011 Van earthquake),38.6,43.5,7.2,7.1 Mw,NaN,NaN,NaN,NaN,NaN,NaN,7.1,Mw


In [12]:
wiki.dtypes

Unnamed: 0              int64
origin                 object
country                object
lat                    object
long                   object
depth_km               object
magnitude              object
secondary_effects      object
pde_shaking_deaths    float64
pde_total_deaths      float64
utsu_total_deaths     float64
emdat_total_deaths     object
deaths                float64
magnitude.1           float64
measure                object
dtype: object

In [20]:
#Need to drop columns with NaNs in lat & long, and index 413, which has ?? in lat/long.
wiki = wiki[wiki['lat'].notna()]
wiki = wiki[wiki['long'].notna()]
wiki = wiki.drop(wiki.index[413])

In [21]:
wiki

,Unnamed: 0,origin,country,lat,long,depth_km,magnitude,secondary_effects,pde_shaking_deaths,pde_total_deaths,utsu_total_deaths,emdat_total_deaths,deaths,magnitude.1,measure
0,0,1900-05-11 17:23,Japan,38.700,141.100,5,7.0 MJMA,NaN,NaN,NaN,NaN,NaN,NaN,7.0,MJMA
1,1,1900-07-12 06:25,Turkey,40.300,43.100,NaN,5.9 Muk,NaN,NaN,NaN,140.0,NaN,140.0,5.9,Muk
2,2,1900-10-29 09:11,Venezuela,11.000,-66.000,0,7.7 Mw,NaN,NaN,NaN,NaN,NaN,NaN,7.7,Mw
3,3,1901-02-15 00:00,China,26.000,100.100,0,6.5 Ms,NaN,NaN,NaN,NaN,NaN,NaN,6.5,Ms
4,4,1901-03-31 07:11,Bulgaria,43.400,28.700,NaN,6.4 Muk,NaN,NaN,NaN,4.0,NaN,4.0,6.4,Muk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1328,1328,2010-02-27 06:34,Chile (see 2010 Chile earthquake),-35.846,-72.719,35,8.8 Mw [18],T,NaN,NaN,NaN,NaN,NaN,8.8,Mw
1334,1334,2011-03-11 14:46,Japan (see 2011 Tōhoku earthquake),38.322,142.369,24.4,9.0 Mw,T,NaN,NaN,NaN,NaN,NaN,9.0,Mw
1336,1336,2011-04-07 14:32,Japan (see April 2011 Miyagi earthquake),38.2,140.0,66.0,7.1 Mw,NaN,NaN,NaN,NaN,NaN,NaN,7.1,Mw
1337,1337,2011-09-18 12:40,India (see 2011 Sikkim earthquake),27.723,88.064,19.7,6.9 Mw,L,NaN,NaN,NaN,NaN,NaN,6.9,Mw


In [22]:
wiki['geometry'] = wiki.apply(lambda x: Point((float(x.long),
                                              float(x.lat))),
                             axis=1)

In [37]:
center = [0, 0]
quake_map = folium.Map(location = center, zoom_start=2)
quake_map

In [38]:
for row_index, row_values in wiki.iterrows():
    loc = [row_values['lat'], row_values['long']]
    pop = str(row_values['origin'])
    marker = folium.Marker(
        location = loc,
        popup = pop)
    
    marker.add_to(quake_map)

quake_map

In [39]:
cluster_quake_map = folium.Map(location = center, zoom_start=2)
marker_cluster = MarkerCluster().add_to(cluster_quake_map)

for row_index, row_values in wiki.iterrows():
    loc = [row_values['lat'], row_values['long']]
    pop = str(row_values['origin'])
    marker = folium.Marker(
        location = loc, 
        popup = pop)
    
    marker.add_to(marker_cluster)
    
cluster_quake_map